# Processing CSV files for SEEDS database
This notebook contains source code for processing different CSV files of SEEDS project and preparing data in a format that allows populating SEEDS app DB.

In [2]:
import pandas as pd
import warnings
import geopandas
import pprint as pp
import json
warnings.filterwarnings('ignore')

## Loading CSV files

The following code read energy dataset files from CSV files and impact data from a json file. 

In [11]:
# New Energy dataset shared in november
nameplate = pd.read_csv('./energy_updated_november23/nameplate_capacity.csv')
storage = pd.read_csv('./energy_updated_november23/storage_capacity.csv')
citizen = pd.read_csv('./energy_updated_november23/citizen_leadership_degree.csv')
acc = pd.read_csv('./energy_updated_november23/deployment_acceleration_rate.csv')
imp = pd.read_csv('./energy_updated_november23/import_dependency.csv')
bio = pd.read_csv('./energy_updated_november23/biofuel_use_rate.csv')
reg_share = pd.read_csv('./energy_updated_november23/max_regional_share.csv')
trans = pd.read_csv('./energy_updated_november23/transmission_capacity.csv')

transmission = pd.read_csv('./energy_updated_november23/transmission_capacity.csv')
energy_tech = pd.read_csv('./energy_updated_november23/energy_supply.csv')
ele_heat = pd.read_csv('./energy_updated_november23/electrification_rate_heat_building.csv')
ele_road = pd.read_csv('./energy_updated_november23/electrification_rate_road_transport.csv')
flow_out = pd.read_csv('./energy_updated_november23/flow_out_sum.csv')

build_heat = pd.read_csv('./energy_updated_november23/generated_building_heat.csv')
district_heat = pd.read_csv('./energy_updated_november23/generated_district_heat.csv')
road_transport = pd.read_csv('./energy_updated_november23/generated_road_transport.csv')

# Impact data from Alex
f = open('./impact_from_alex.json')
impact = json.load(f)

In [4]:
energy_tech['techs'].unique()

array(['biofuel_supply', 'el_import', 'existing_pv', 'existing_wind',
       'h2_import', 'hydro_reservoir', 'hydro_run_of_river',
       'open_field_pv', 'roof_mounted_pv', 'waste_supply',
       'wind_offshore', 'wind_onshore'], dtype=object)

In [5]:
label_mapping = {}
label_mapping['land_occupation'] = 'agricultural land occupation (LOP)'
label_mapping['surplus_ore'] = 'surplus ore potential (SOP)'
label_mapping['global_warming'] = 'global warming potential (GWP1000)'
label_mapping['water_consumption'] = 'water consumption potential (WCP)'
label_mapping['freshwater_eutrophication'] = 'freshwater eutrophication potential (FEP)'

In [15]:
# Unique type of power generating technology after excluding electrolysis
power_tech = [item  for item in list(nameplate.techs.unique())]


## Preparing data for populating the following tables in DB
The SEEDs DB schema is given below. To populate SEEDs DB, we will first start with creating a Scenario table which will store one etry per scenario. 

`Table: Scenario`

Columns: 

    power_capacity 
    storage_capacity
    community_infrastructure
    implementation_pace
    import_dependency
    bio_fuel
    battery
    max_regional_share
    
    ## computed by summing up data across all location for each scenario
    on_shore_wind
    off_shore_wind
    open_field_pv
    roof_mounted_pv
    hydro_run_river
    
    ## computed by summing up data across different technologies ( for generation as well as 
        storage) for each scenario
    land_occupation
    surplus_ore
    global_warming
    water_consumption
    freshwater_eutrophication
    
    

`Table: EnergySupply`

Columns:
    
    scenario
    location
    tech_type
    energy_supply
    
    
`Table: Electrification`

Columns:

    scenario
    location
    carrier_type
    ele_rate
    
    
`Table: Transmission`

Columns:

    scenario
    from_location
    to_location
    transmission
    
    
`Table: FlowOut`

Columns:

    scenario
    tech_type
    location
    carriers
    flow_out_sum


`Table: GeneratedHeat`

Columns:

    scenario
    tech_type
    location
    heat_type
    heat
    

`Table: GeneratedTransport`

Columns:

    scenario
    tech_type
    location
    transport

Each scenario has different power generation technologies at different locations. Therefore, to compute a total of power generation as well as battery storage, we would perform groupby operation on scenario id (spore).

In [8]:
# summing total power generation for selected technology type
pw_nameplate = nameplate.loc[nameplate['techs'].isin(power_tech),:]
total_power = pw_nameplate.groupby(['spores']).sum().reset_index()

# summing total power storage for selected technology type
pw_storage = storage.loc[storage['techs'].isin(power_tech),:]
total_storage = storage.groupby(['spores']).sum().reset_index()

battery_storage = storage.loc[storage['techs'] == 'battery',:]
total_battery = battery_storage.groupby(['spores']).sum().reset_index()

In [21]:
scenario.shape

(261, 22)

### Preparing partial Scenario table

In [9]:
# Scenario table with partial columns dataframe
scenario_partial = pd.DataFrame(columns=['id','infra','pace','import','power',
                                         'storage',
                                         'bio','max_regional_share','battery',
                                         'land_occupation',
                                         'surplus_ore',
                                         'global_warming',
                                         'water_consumption',
                                         'freshwater_eutrophication'])

In [12]:
scenario_ids = sorted(acc.spores.unique())

for spore in scenario_ids:
    scenario_id = spore
    scenario_impact = impact[str(spore)]
    
    infrastructure = citizen.loc[citizen['spores'] == spore,'citizen_leadership_degree'].to_list()[0]
    deployment_rate = acc.loc[acc['spores'] == spore,'deployment_acceleration_rate'].to_list()[0]
    import_dependency = imp.loc[imp['spores'] == spore,'import_dependency'].to_list()[0]
    power_capacity = total_power.loc[total_power['spores'] == spore,'nameplate_capacity'].to_list()[0]
    storage_capacity = total_storage.loc[total_storage['spores'] == spore,'storage_capacity'].to_list()[0]
    battery_storage = total_battery.loc[total_battery['spores'] == spore,'storage_capacity'].to_list()[0]
    bio_fuel = bio.loc[bio['spores'] == spore,'biofuel_use_rate'].to_list()[0]
    max_regional_share = reg_share.loc[bio['spores'] == spore,'max_regional_share'].to_list()[0]
    
    scenario_partial = scenario_partial.append({'id':spore,
                                                'infra':infrastructure,
                                                'pace':deployment_rate,
                                                'import':import_dependency,
                                                'power':power_capacity,
                                                'storage':storage_capacity,
                                                'bio':bio_fuel,
                                                'max_regional_share':max_regional_share,
                                                'battery':battery_storage,
                                                'land_occupation':scenario_impact[label_mapping['land_occupation']],
                                                'surplus_ore':scenario_impact[label_mapping['surplus_ore']],
                                                'global_warming':scenario_impact[label_mapping['global_warming']],
                                                'water_consumption':scenario_impact[label_mapping['water_consumption']],
                                                'freshwater_eutrophication':scenario_impact[label_mapping['freshwater_eutrophication']]
                                               },ignore_index=True)
    """
    print('Object:')
    print('   infrastructure:',infrastructure)
    print('   deployment_rate:',deployment_rate)
    print('   import_dependency:',import_dependency)
    print('   power_capacity:',power_capacity)
    print('   storage_capacity:',storage_capacity)
    print('   biofule_use_rate:',bio_fuel)
    """

In [14]:
#scenario_partial.to_csv('./Processed_november_dataset/Scenario_with_impact_alex_data.csv',index=False)

### Location objects

In [15]:
merged = geopandas.read_file('../portugal_regions.geojson')
merged.set_index('index',inplace=True)

### Preparing technology generation and storage data
Here, we will process CSV files to get data for two tables: TechGeneration and TechStorage

    Table: TechGeneration
    Columns
        location
        scenario
        technology_type
        energy_generation
    
    
    Table: TechStorage
    Columns
        scenario
        location
        technology_type
        energy_storage


In [19]:
tech_gen = pd.DataFrame(columns=['scenario','location','tech_type','value'])
tech_sto = pd.DataFrame(columns=['scenario','location','tech_type','value'])

In [20]:
battery = nameplate.loc[nameplate['techs'] == 'battery',:]

In [21]:
def getTechGenerationRecords(scenario_id,tech_type,loc = None):
    """
    This function returns location wise value/values of generation capacity for specified technology
    
    """

    sc_dataset = nameplate.loc[nameplate['spores'] == scenario_id,:]
    tech_data = sc_dataset.loc[sc_dataset['techs'] == tech_type,:][['spores','locs','nameplate_capacity']]
    
    if loc:
        return tech_data.loc[tech_data['locs'] == loc,:]['nameplate_capacity'].to_list()[0]
    else:
        return tech_data['locs'].to_list(),tech_data['nameplate_capacity'].to_list()

In [22]:
def getTechStorageRecords(scenario_id,tech_type,loc = None):
    """
    This function returns location wise value/values of storage capacity for specified technology
    
    """
    sc_dataset = storage.loc[storage['spores'] == scenario_id,:]
    tech_data = sc_dataset.loc[sc_dataset['techs'] == tech_type,:][['spores','locs','storage_capacity']]
    
    if loc:
        return tech_data.loc[tech_data['locs'] == loc,:]['storage_capacity'].to_list()[0]
    else:
        return tech_data['locs'].to_list(),tech_data['storage_capacity'].to_list()

In [23]:
# Preparing data for TechGeneration Table
for s in scenario_partial.index:
    for tech in power_tech:
        locs,values = getTechGenerationRecords(s,tech)
        for index, loc in enumerate(locs):
            record = {'scenario':s,'location':loc}
            value = values[index]
            record['tech_type'] = tech
            record['value'] = value
            tech_gen = tech_gen.append(record,ignore_index=True) 

In [24]:
tech_gen.to_csv('./Processed_november_dataset/TechGeneration.csv',index=False)

In [25]:
# TechStorage Table
for s in scenario_partial.index:
    for tech in power_tech:
        locs,values = getTechStorageRecords(s,tech)
        for index, loc in enumerate(locs):
            record = {'scenario':s,'location':loc}
            value = values[index]
            record['tech_type'] = tech
            record['value'] = value
            tech_sto = tech_sto.append(record,ignore_index=True) 

In [26]:
tech_sto.to_csv('./Processed_november_dataset/TechStorage.csv',index=False)

### Extending partial Scenario table by including remaining columns
We will now use the other dataframes to extend Scenario table which was previously created partially. We will add now total of energy generation for each scenario across different technologies. 

In [27]:
scenario_partial = pd.read_csv('./Processed_november_dataset/Scenario_with_impact_alex_data.csv')
tech_gen = pd.read_csv('./Processed_november_dataset/TechGeneration.csv')
tech_sto = pd.read_csv('./Processed_november_dataset/TechStorage.csv')

In [28]:
# columns to sum for each scenario 
results_columns = ['wind_onshore',
                   'wind_offshore',
                   'open_field_pv',
                   'roof_mounted_pv',
                   'hydro_run_of_river','chp_hydrogen']

# extracting first results columns values

results_df = tech_gen.loc[tech_gen['tech_type'].isin(results_columns),:]
results_ag = results_df.groupby(by=['scenario','tech_type']).sum()

In [29]:
remaining_scenario_columns = ['scenario'] + results_columns 

remaining_scenario = pd.DataFrame(columns = remaining_scenario_columns)

In [30]:
battery_sto = tech_sto.loc[tech_sto.tech_type=='battery',:].groupby(by=['scenario','tech_type']).sum()
battery_sto.loc[0,:].to_dict()

{'value': {'battery': 3.123265754514433e-07}}

In [31]:
scenario_ids = sorted(scenario_partial.id.unique())

In [32]:
for scenario in scenario_ids:
    record = {'scenario':scenario}
    temp_tech = results_ag.loc[scenario,:].to_dict()['value']
    
    for key,value in temp_tech.items():
        record[key] = value

    temp_df = pd.DataFrame(record,index=[0])
    
    remaining_scenario = pd.concat([remaining_scenario,temp_df])

remaining_scenario.reset_index(inplace=True)

In [51]:
"""
The minimum and maximum values for each attribute of the Scenario table. 
These values are used for scaling purposes.
"""

param_config = {'battery': {'max': 0.0444797482359873, 'min': 2.6183049409038122e-08},
                'bio': {'max': 1.0009737770934894, 'min': 4.134278090390732e-06},
                'freshwater_eutrophication': {'max': 1447977268.6580737,
                                              'min': 40539809.033321954},
                'global_warming': {'max': 3550705057804.7417, 'min': 89357470400.92726},
                'hydro_run_of_river': {'max': 1.6155, 'min': 1.6155},
                'import': {'max': 0.2, 'min': 0},
                'import_dep': {'max': 0.1561112495614454, 'min': 0.0003012119877652},
                'infra': {'max': 1.0, 'min': 0},
                'land_occupation': {'max': 216513993115, 'min': 20280814757},
                'open_field_pv': {'max': 125, 'min': 0},
                'pace': {'max': 13.096833998642731, 'min': 3.729211861614511},
                'power': {'max': 159.63839465280677, 'min': 40.8581235544374},
                'roof_mounted_pv': {'max': 26, 'min': 0},
                'scenario': {'max': 260.0, 'min': 0.0},
                'storage': {'max': 6.303784879568456, 'min': 5.898367317341074},
                'surplus_ore': {'max': 655458310608.2251, 'min': 9926424157.968464},
                'water_consumption': {'max': 61979898600.96115, 'min': 859061013.8176432},
                'wind_offshore': {'max': 13, 'min': 0},
                'wind_onshore': {'max': 45, 'min': 0}}

RangeIndex(start=0, stop=261, step=1)

In [33]:
scenario = scenario_partial.join(remaining_scenario)

In [34]:
import numpy as np
scenario.to_csv('./Processed_november_dataset/Scenario_with_impact_alex_data_ele_generation.csv',index=False)
scenario = scenario.replace({np.nan:None})

In [20]:
scenario = pd.read_csv('./Processed_november_dataset/Scenario_with_impact_alex_data_ele_generation.csv')
scenario['battery'].describe()


count    2.610000e+02
mean     2.255985e-03
std      2.440277e-03
min      5.575166e-08
25%      2.774109e-04
50%      1.294443e-03
75%      3.641627e-03
max      1.111376e-02
Name: battery, dtype: float64

# Script to populate dataset

In [ ]:
We now use difference CSV files -which are generated before- to populate data into SEEDs database. 

In [46]:
scenario = pd.read_csv('./Processed_november_dataset/Scenario_with_impact_alex_data_ele_generation.csv')
tech_gen = pd.read_csv('./Processed_november_dataset/TechGeneration.csv')
tech_sto = pd.read_csv('./Processed_november_dataset/TechStorage.csv')


In [38]:
scenario.columns

Index(['id', 'infra', 'pace', 'import', 'power', 'storage', 'bio',
       'max_regional_share', 'battery', 'land_occupation', 'surplus_ore',
       'global_warming', 'water_consumption', 'freshwater_eutrophication',
       'index', 'scenario', 'wind_onshore', 'wind_offshore', 'open_field_pv',
       'roof_mounted_pv', 'hydro_run_of_river', 'chp_hydrogen'],
      dtype='object')

In [19]:
scenario['import_dep'] = scenario['import']
con = {}

In [20]:
for col in scenario.columns:
    #print(col)
    con[col] = {}
    #print(' Min:',scenario[col].min())
    #print(' Max:',scenario[col].max())
    con[col]['min'] = scenario[col].min()
    con[col]['max'] = scenario[col].max()
    print(' ')

id
 Min: 0.0
 Max: 260.0
 
infra
 Min: 0.1881200008807587
 Max: 1.0
 
pace
 Min: 3.729211861614511
 Max: 13.096833998642731
 
import
 Min: 0.0003012119877652
 Max: 0.1561112495614454
 
power
 Min: 44.0194718081856
 Max: 163.17747814310155
 
storage
 Min: 5.898367317341074
 Max: 6.303784879568456
 
bio
 Min: 4.134278090390732e-06
 Max: 1.0009737770934894
 
battery
 Min: 2.6183049409038122e-08
 Max: 0.0444797482359873
 
land_occupation
 Min: 20280814758.748016
 Max: 216513993113.64584
 
surplus_ore
 Min: 9926424157.968464
 Max: 655458310608.2251
 
global_warming
 Min: 89357470400.92726
 Max: 3550705057804.7417
 
water_consumption
 Min: 859061013.8176432
 Max: 61979898600.96115
 
freshwater_eutrophication
 Min: 40539809.033321954
 Max: 1447977268.6580737
 
index
 Min: 0
 Max: 0
 
scenario
 Min: 0.0
 Max: 260.0
 
wind_onshore
 Min: 1.5571043872453
 Max: 44.36655376398457
 
wind_offshore
 Min: 0.003085515351798
 Max: 12.2881549166774
 
open_field_pv
 Min: 2.231546741630743
 Max: 121.8423242

In [161]:
import json
# Save parameters min-max values to use in SEEDs web app
with open('config.json','w+') as f:
    json.dump(con,f)

In [52]:
for s in scenario.itertuples():
    print('Scenario id:',int(s.id))
    s_record = dict(power_capacity = s.power,
            storage_capacity = s.storage,
            community_infrastructure = s.infra,
            import_dependency = s.import_dep,
            implementation_pace = s.pace,
            bio_fuel = s.bio,
            wind_onshore = s.wind_onshore,
            wind_offshore = s.wind_offshore,
            open_field_pv = s.open_field_pv,
            roof_mounted_pv = s.roof_mounted_pv,
            hydro_run_of_river = s.hydro_run_of_river,
            global_warming = s.global_warming,
            land_occupation = s.land_occupation,
            surplus_ore = s.surplus_ore,
            water_consumption = s.water_consumption,
            freshwater_eutrophication = s.freshwater_eutrophication,
            battery = s.battery
            )
    
    #pp.pprint(s_record)
    
    tech_gen_s = tech_gen[tech_gen['scenario'] == s.id]
    tech_sto_s = tech_sto[tech_sto['scenario'] == s.id]
    
    for tech_gen_record  in tech_gen_s.itertuples():
        print({'scenario':s.id,
              'location':tech_gen_record.location,
              'technology_type':tech_gen_record.tech_type,
              'energy_generation':tech_gen_record.value})
        
        
    for tech_sto_record  in tech_sto_s.itertuples():
        print({'scenario':s.id,
              'location':tech_sto_record.location,
              'technology_type':tech_sto_record.tech_type,
              'energy_storage':tech_sto_record.value})
        
    # Populating electrification tables
    ele_heat_s = ele_heat[ele_heat['spores'] == s.id]
    ele_road_s = ele_road[ele_road['spores'] == s.id]
    
    for ele_heat_record  in ele_heat_s.itertuples():
        print({'scenario':s.id,
              'location':ele_heat_s.locs,
              'carriers_type':'heat',
              'electrification_rate':ele_heat_s.electrification_rate_heat_building})
        
    for ele_road_record  in ele_road_s.itertuples():
        print({'scenario':s.id,
              'location':ele_road_s.locs,
              'carriers_type':'transport',
              'electrification_rate':ele_road_s.electrification_rate_road_transport})
    
    #Populating energy supply table
    energy_tech_s = energy_tech[energy_tech['spores'] == s.id]
    for energy_tech_record  in energy_tech_s.itertuples():
        print({'scenario':s.id,
              'location':energy_tech_record.locs,
              'technology_type':energy_tech_record.techs,
              'energy_supply':energy_tech_record.energy_supply})
        
    # Populating transmission table
    transmission_s = transmission[transmission['spores'] == s.id]
    for transmission_record  in transmission_s.itertuples():
        print({'scenario':s.id,
              'from_location':transmission_record.exporting_region,
              'from_location':transmission_record.importing_region,
              'transmission_capacity':transmission_record.transmission_capacity})
        
    
    # Populating GeneratedHeat table
    build_heat_s = build_heat[build_heat['spores'] == s.id]
    district_heat_s = district_heat[district_heat['spores'] == s.id]
    
    for build_heat_record  in build_heat_s.itertuples():
        print({'scenario':s.id,
              'tech_type':build_heat_record.techs,
              'location':build_heat_record.locs,
              'heat_type':'building',
              'heat':build_heat_record.generated_building_heat})
        
    for dis_heat_record  in district_heat_s.itertuples():
        print({'scenario':s.id,
              'tech_type':dis_heat_record.techs,
              'location':dis_heat_record.locs,
              'heat_type':'district',
              'heat':dis_heat_record.generated_district_heat})
        
    # Populating GeneratedTransport table 
    road_transport_s = road_transport[road_transport['spores'] == s.id]
    for road_transport_record  in road_transport_s.itertuples():
        print({'scenario':s.id,
              'tech_type':road_transport_record.techs,
              'location':road_transport_record.locs,
              'transport':road_transport_record.generated_road_transport
              })
        
    # Populating FlowOut table 
    flow_out_s = flow_out[flow_out['spores'] == s.id]
    for flow_out_record  in flow_out_s.itertuples():
        print({'scenario':s.id,
              'tech_type':flow_out_record.techs,
              'location':flow_out_record.locs,
              'carriers':flow_out_record.carriers,
              'flow_out_sum':flow_out_record.flow_out_sum
              })
        
    break

Scenario id: 0
{'scenario': 0.0, 'location': 'PRT_1', 'technology_type': 'chp_biofuel_extraction', 'energy_generation': 6.060198178326705e-05}
{'scenario': 0.0, 'location': 'PRT_2', 'technology_type': 'chp_biofuel_extraction', 'energy_generation': 5.05138003229184e-05}
{'scenario': 0.0, 'location': 'PRT_1', 'technology_type': 'chp_hydrogen', 'energy_generation': 2.460120553517743e-05}
{'scenario': 0.0, 'location': 'PRT_2', 'technology_type': 'chp_hydrogen', 'energy_generation': 3.286773159307333e-05}
{'scenario': 0.0, 'location': 'PRT_1', 'technology_type': 'chp_methane_extraction', 'energy_generation': 1.00550453288186e-05}
{'scenario': 0.0, 'location': 'PRT_2', 'technology_type': 'chp_methane_extraction', 'energy_generation': 1.0773054990210089e-05}
{'scenario': 0.0, 'location': 'PRT_1', 'technology_type': 'chp_wte_back_pressure', 'energy_generation': 0.0963488954942974}
{'scenario': 0.0, 'location': 'PRT_1', 'technology_type': 'chp_wte_back_pressure', 'energy_generation': 5.63951205